In [8]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
from torchvision.models import ResNet50_Weights
from torchvision.transforms import transforms

In [11]:
# Prefer to load as npy so we can use ToTensor
features = np.load('./data/raw/features.npy') # [num, 299, 299]
labels = np.load('./data/raw/labels.npy') # [num, 4]

In [12]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))  # ResNet50 works best on [224, 224]
])

features = torch.stack([preprocess(f) for f in features])
labels = torch.tensor(labels).type(torch.float)

# Print the data types
print(f'Features dtype: {features.dtype}')
print(f'Features shape: {features.shape}')
print(f'Labels dtype: {labels.dtype}')
print(f'Labels shape: {labels.shape}')

# Create a TensorDataset
dataset = TensorDataset(features, labels)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True) # Stratify?

Features dtype: torch.float32
Features shape: torch.Size([21165, 1, 224, 224])
Labels dtype: torch.float32
Labels shape: torch.Size([21165, 4])


In [13]:
# Load the pretrained ResNet50 model
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Modify the first convolutional layer to accept single-channel input, other params are kept as ResNet50 defaults (padding=3 ensures output shape is [150, 150])
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the final fully connected layer to match the number of classes (4)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# Move the model to GPU if available
device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
print("Device is set to:", device)
model = model.to(device, torch.float)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Device is set to: mps


In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, torch.max(labels, 1)[1])

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

print('Training complete')